In [1]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [18]:
# getting Twitter Dev keys and tokens from txt file
with open('TwitterKeysTokens.txt') as f:
    content = f.read().splitlines()

## First lets define the topic tou want to enquire about...

### for example... 'marmite' or 'Boris Johnson'.

Do keep in mind that TextBlob is not very good at detecting sarcasm or nuanced topics

In [27]:
# Run this cell to set your topic
topic = input()

In [16]:
class TwitterClient(object):
    '''
    generic Twitter Class for sentiment Analysis.

    '''
    def __init__(self):
        '''
        Class constructor or initialisation method

        '''

        # keys and tokens from Twitter Dev method
        consumer_key = content[0]
        consumer_secret = content[1]
        access_token = content[2]
        access_token_secret = content[3]

        # attempts authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and access token secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweetpy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        
        except:
            print('ERROR:  authentication failed')
        
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special
        characters using simple regex statements.

        '''

        return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)', ' ', tweet).split())
    
    def get_tweet_sentiment(self, tweet):
        '''
        utility function to classify sentiment of passed tweet using
        textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'


    def get_tweets(self, query, count=1000):
        '''
        main function to fetch tweets and parse them
        '''
        # empty list to store parse tweets
        tweets = []

        try:
            #  call twitter api to fetch tweets
            fetched_tweets = self.api.search(q=query, count=count)

            # parsing tweets one by one:
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
                
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is only appended once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets 
            return tweets

        except tweepy.TweepError as e:
            # print error, if any:
            print('ERROR: ' + str(e))

In [29]:
def main():
    # creating object of twitterClient Class
    api = TwitterClient()
    # calling function to get tweets change query topic
    tweets = api.get_tweets(query=topic, count=200)

    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print('Positive Tweets Percentage: {} %'.format(100*len(ptweets)/len(tweets)))

    # selecting negative tweets:
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print('Negative Tweets Percentage: {} %'.format(100*len(ntweets)/len(tweets)))

    # neutral tweets
    neutraltweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']
    # percentage of neutral tweets:
    print('Neutral Tweets Percentage: {} %'.format(100*len(neutraltweets)/len(tweets)))

    # printing first 5 positive tweets
    print('\n\nPositive Tweets:')
    for tweet in ptweets[:100]:
        print(tweet['text'])

    # printing first 5 negative tweets:
    print('\n\nNegative Tweets:')
    for tweet in ntweets[:100]:
        print(tweet['text'])

if __name__ == '__main__':
    # calling main function
    main()

Positive Tweets Percentage: 40.69767441860465 %
Negative Tweets Percentage: 9.30232558139535 %
Neutral Tweets Percentage: 50.0 %


Positive Tweets:
@TripeUK @UpTheWorkers I think ur right hadn’t noticed - a lot easier to get out of jar ——- I love marmite with avo… https://t.co/OgMXg63e7e
RT @Myriam_Liebl: Les porcs là sont cute de ouf awwwn. Regardez comment ils dorment entassés. 🤣
Mais ils vont malheureusement finir dans la…
Ready to try a sound-taste experiment @BBCRadio4 The Kitchen Cabinet #bbctkc ? Originally run ⁦@IP_SAS⁩ Lab by Russ… https://t.co/Ul25SBEPHu
@TripeUK Agreed - still love it though #marmite 💕
@iBreezeblock @pibarrister I stock piled early in lockdown when someone on Twitter whispered the chance we could ha… https://t.co/GC7nmPCvFS
@SwearyBerry81 @J2onyabike @CarriePurdom @2_Wheeled_Wolf @gazza_d Exactly! He broke his taste buds and now thinks Marmite is somehow edible
@TripeUK Null hypothesis: Marmite is the same but you are getting older and have your heating set 